In [1]:
%run Old_files/From1._toBottleNeckDetective.ipynb #change model source here

No objective coefficients in model. Unclear what should be optimized


We will investigate each step of the psilocybin production pathway, to see why our production rate is so low, and always the same, regardless of media optimization, gene target modulation, etc.

Within this notebook, we are testing individual steps as demand functions, and set the production of the targeted metabolite as the model's objective function.
   
   
Tryptamine production:

In [2]:
with model:
    model.add_boundary(model.metabolites.tryp_c, type='demand')
    target = model.reactions.DM_tryp_c
    model.objective = target
    psi_production = model.optimize().objective_value
    print("Max. tryptamine production [mmol gDW^-1 h^-1]:", psi_production)

Max. tryptamine production [mmol gDW^-1 h^-1]: 0.212352921194493


We still have flux

4-Hydroxytryptamine:

In [3]:
with model:
    model.add_boundary(model.metabolites._4htryp_c, type='demand')
    target = model.reactions.DM__4htryp_c
    model.objective = target
    psi_production = model.optimize().objective_value
    print("Max. 4-hydroxytryptamine production [mmol gDW^-1 h^-1]:", psi_production)

Max. 4-hydroxytryptamine production [mmol gDW^-1 h^-1]: 0.212352921194493


We still have flux.

Norbaeocystin:

In [4]:
with model:
    model.add_boundary(model.metabolites.nbc_c, type='demand')
    target = model.reactions.DM_nbc_c
    model.objective = target
    psi_production = model.optimize().objective_value
    print("Max. norbaeocystin production [mmol gDW^-1 h^-1]:", psi_production)

Max. norbaeocystin production [mmol gDW^-1 h^-1]: 0.212352921194493


We still have flux.

Baeocystin:

In [5]:
with model:
    model.add_boundary(model.metabolites.baecyst_c, type='demand')
    target = model.reactions.DM_baecyst_c 
    model.objective = target
    psi_production = model.optimize().objective_value
    print("Max. baeocystin production [mmol gDW^-1 h^-1]:", psi_production)
    print("Methionine->SAM [mmol-psi / mmol-gl]:", model.reactions.METAT.flux)

Max. baeocystin production [mmol gDW^-1 h^-1]: 0.0
Methionine->SAM [mmol-psi / mmol-gl]: 0.0


We have identified the bottleneck. It appears related baeocystin production from norbaeocystein using SAM and SAH as cofactors. Below we will investigate further.

First, SAH is added as an additional demand function. It is seen later that there are few reactions present to degrade SAH, so otherwise it is accumulating.

In [6]:
with model:
    model.add_boundary(model.metabolites.baecyst_c, type='demand')
    model.add_boundary(model.metabolites.ahcys_c, type='demand')
    target = model.reactions.DM_baecyst_c  
    model.objective = target
    psi_production = model.optimize().objective_value
    print("Max. baeocystin production [mmol gDW^-1 h^-1]:", psi_production)
    print("Methionine->SAM [mmol-psi / mmol-gl]:", model.reactions.METAT.flux)

Max. baeocystin production [mmol gDW^-1 h^-1]: 0.000113859976741657
Methionine->SAM [mmol-psi / mmol-gl]: 0.000113859976741657


Now we see a very low production rate, but higher than zero and the original production rate of 5.8e-5., showing that the reaction is proceeding. SAM production seems to be influencial on the production rate, so we will investigate further.

In [7]:
with model:
    model.add_boundary(model.metabolites.baecyst_c, type='demand')
    target = model.reactions.DM_baecyst_c
    print("Max. psi production [mmol gDW^-1 h^-1]:",model.optimize().objective_value)
    for x in model.reactions.query('amet_c','reaction'):
        print("%s : %s" % (x.id, x.reaction))

Max. psi production [mmol gDW^-1 h^-1]: 0.0
AMAOTr : 8aonn_c + amet_c --> amob_c + dann_c
AMMQLT8 : 2dmmql8_c + amet_c --> ahcys_c + h_c + mql8_c
BTS4 : amet_c + dtbt_c + s_c --> btn_c + daD_5_c + h_c + met_L_c
CPPPGO2 : 2.0 amet_c + cpppg3_c --> 2.0 co2_c + 2.0 daD_5_c + 2.0 met_L_c + pppg9_c
LIPOS : 4fe4s_c + 2.0 amet_c + h_c + nad_c + octapb_c --> 2fe2s_c + 2.0 daD_5_c + 2.0 fe2_c + lipopb_c + 2.0 met_L_c + nadh_c
OMBZLM : 2ombzl_c + amet_c --> 2ommbl_c + ahcys_c + h_c
TYRL : amet_c + nadph_c + tyr_L_c --> 4crsol_c + daD_5_c + dhgly_c + h_c + met_L_c + nadp_c
UPP3MT : 2.0 amet_c + uppg3_c --> 2.0 ahcys_c + dscL_c + h_c
METAT : atp_c + h2o_c + met_L_c --> amet_c + pi_c + ppi_c
METTRS : atp_c + met_L_c + trnamet_c --> amp_c + mettrna_c + ppi_c
SOL_POOL : 0.12386 accoa_c + 0.099086 amet_c + 0.000887 btn_c + 0.099086 chor_c + 0.074314 coa_c + 0.013872 malcoa_c + 0.79269 nad_c + 0.019817 nadh_c + 0.049543 nadp_c + 0.14863 nadph_c + 0.099086 pydx5p_c + 0.043598 succoa_c + 0.024558 udcpdp_

amet_c is SAM and ahcys_c is SAH.

We can see that one source of SAH is METAT, producing SAH from Methionine.

In [8]:
print(model.reactions.METAT.id,model.reactions.METAT.bounds,model.reactions.METAT.reaction) #closer look

METAT (0.0, 0.000113859976741657) atp_c + h2o_c + met_L_c --> amet_c + pi_c + ppi_c


The observed production of baeocystin corresponds to the upper bound for production of SAM from methionine. 

The higher bound is quite low, so it is thought that increasing the bound would allow a higher product flux.

In [9]:
with model:
    model.add_boundary(model.metabolites.baecyst_c, type='demand')
    model.add_boundary(model.metabolites.ahcys_c, type='demand')
    target = model.reactions.DM_baecyst_c 
    model.reactions.METAT.bounds=-1000,1000
    model.objective = target
    psi_production = model.optimize().objective_value
    print("Max. baeocystin production [mmol gDW^-1 h^-1]:", psi_production)
    print("Methionine->SAM [mmol-psi / mmol-gl]:", model.reactions.METAT.flux)

Max. baeocystin production [mmol gDW^-1 h^-1]: 0.20166999999999996
Methionine->SAM [mmol-psi / mmol-gl]: 0.20166999999999996


The two reactions go hand in hand, and baeocystin production now matches norbaeocystin.

It is hypothesized that adding methionine to the media would increase the reaction flux.

In [10]:
with model:
    model.add_boundary(model.metabolites.baecyst_c, type='demand')
    model.add_boundary(model.metabolites.ahcys_c, type='demand')
    target = model.reactions.DM_baecyst_c
    model.reactions.METAT.bounds=-1000,1000
    model.reactions.EX_met_L_e.bounds=-1000,1000 #artificially expand
    model.objective = target
    psi_production = model.optimize().objective_value
    print("Max. baeocystin production [mmol gDW^-1 h^-1]:", psi_production)
    print("Methionine->SAM [mmol-psi / mmol-gl]:", model.reactions.METAT.flux)

Max. baeocystin production [mmol gDW^-1 h^-1]: 0.212352921194493
Methionine->SAM [mmol-psi / mmol-gl]: 0.212352921194493


This increased the production rate as expected.

Now the product, psilocybin, is tested.

In [11]:
with model:
    model.add_boundary(model.metabolites.psi_c, type='demand')
    model.add_boundary(model.metabolites.ahcys_c, type='demand')
    target = model.reactions.DM_psi_c
    model.reactions.METAT.bounds=-1000,1000
    model.objective = target
    psi_production = model.optimize().objective_value
    print("Max. psilocybin production [mmol gDW^-1 h^-1]:", psi_production)
    print("Methionine->SAM [mmol-psi / mmol-gl]:", model.reactions.METAT.flux)

Max. psilocybin production [mmol gDW^-1 h^-1]: 0.11524000000000006
Methionine->SAM [mmol-psi / mmol-gl]: 0.23048000000000013


The production rate is seen too fall, as the production of SAM cannot keep up.

Adding methionine to the medium:

In [12]:
with model:
    model.add_boundary(model.metabolites.psi_c, type='demand')
    model.add_boundary(model.metabolites.ahcys_c, type='demand')
    target = model.reactions.DM_psi_c
    model.reactions.METAT.bounds=-1000,1000
    model.reactions.EX_met_L_e.bounds=-1000,1000
    model.objective = target
    psi_production = model.optimize().objective_value
    print("Max. psilocybin production [mmol gDW^-1 h^-1]:", psi_production)
    print("Methionine->SAM [mmol-psi / mmol-gl]:", model.reactions.METAT.flux)

Max. psilocybin production [mmol gDW^-1 h^-1]: 0.1761877527166955
Methionine->SAM [mmol-psi / mmol-gl]: 0.352375505433391


This is a very desirable production rate, however, it is not naturally able to occur in the Corynebacterium glutamicum. The original bounds of the reactions in the model are based on 13C-tracer experimental fluxes, and modification to the microorganism would be necessary. 

It seems SAM still limits the reaction to some extent, as it is used in both baeocystin and psilocybin production.

Upregulating the genes that encode for SAM production would be vital for the production of psilocybin, and including methionine in the medium would be a good idea. 